![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [58]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

As with most data science projects, you will need to pre-process the data provided, in this case, a csv file called `rental_info.csv`. Specifically, you need to:
- Read in the csv file `rental_info.csv` using pandas.

In [59]:
df = pd.read_csv('rental_info.csv')
df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [61]:
df.describe().round(2)

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
count,15861.00,15861.00,15861.00,15861.00,15861.00,15861.0,15861.0,15861.00,15861.0,15861.00,15861.00,15861.00
mean,4.22,2006.89,2.94,114.99,20.22,0.2,0.2,0.22,0.2,23.36,14832.84,11.39
std,2.36,2.03,1.65,40.11,6.08,0.4,0.4,0.42,0.4,23.50,9393.43,10.01
min,0.99,2004.00,0.99,46.00,9.99,0.0,0.0,0.00,0.0,0.98,2116.00,0.98
25%,2.99,2005.00,0.99,81.00,14.99,0.0,0.0,0.00,0.0,8.94,6561.00,0.98
50%,3.99,2007.00,2.99,114.00,20.99,0.0,0.0,0.00,0.0,15.92,12996.00,8.94
75%,4.99,2009.00,4.99,148.00,25.99,0.0,0.0,0.00,0.0,24.90,21904.00,24.90
max,11.99,2010.00,4.99,185.00,29.99,1.0,1.0,1.00,1.0,143.76,34225.00,24.90


- Create a column named `"rental_length_days"` using the columns `"return_date"` and `"rental_date"`, and add it to the pandas DataFrame. This column should contain information on how many days a DVD has been rented by a customer.

In [62]:
df['return_date'] = pd.to_datetime(df['return_date'])
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['rental_length_days'] = (df['return_date'] - df['rental_date']).dt.days
print(df['rental_length_days'].head())

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64


- Create two columns of dummy variables from `"special_features"`, which takes the value of 1 when:
    - The value is `"Deleted Scenes"`, storing as a column called `"deleted_scenes"`.
    - The value is `"Behind the Scenes"`, storing as a column called `"behind_the_scenes"`.

In [63]:
df['deleted_scenes'] = df['special_features'].str.contains('Deleted Scenes').astype(int)
df['behind_the_scenes'] = df['special_features'].str.contains('Behind the Scenes').astype(int)
df[['deleted_scenes','behind_the_scenes']].head()

,deleted_scenes,behind_the_scenes
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


- Make a pandas DataFrame called `X` containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.

In [64]:
print('\n'.join(df.columns))

rental_date
return_date
amount
release_year
rental_rate
length
replacement_cost
special_features
NC-17
PG
PG-13
R
amount_2
length_2
rental_rate_2
rental_length_days
deleted_scenes
behind_the_scenes


In [65]:
drop_columns = ['rental_date', 'return_date', 'special_features', 'rental_length_days']
X = df.drop(drop_columns, axis=1)

- Choose the `"rental_length_days"` as the target column and save it as a pandas Series called `y`.

In [66]:
y = df['rental_length_days']

Following the preprocessing you will need to:
- Split the data into `X_train`, `y_train`, `X_test`, and `y_test` train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.
- **Set random_state to 9** whenever you use a function/method involving randomness, for example, when doing a test-train split.

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)

**Recommend a model yielding a mean squared error (MSE) less than 3 on the test set**
- Save the model you would recommend as a variable named best_model, and save its MSE on the test set as `best_mse`.

In [68]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)
mse_lm = mean_squared_error(y_pred, y_test)
print(mse_lm)

2.9417238646975883


In [69]:
best_model = lm
best_mse = mse_lm